# Using concepts

This notebook will show you how to use concepts from python.

For more information on concepts, see our [Concepts](https://docs.lilacml.com/concepts/concepts.html) guide.


In [2]:
%load_ext autoreload
%autoreload 2
import lilac as ll

ll.set_project_dir('./data')

concept_namespace = 'lilac'
concept_name = 'language-model-reference'
embedding_name = 'gte-small'

/Users/nikhil/Code/lilac/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get the concept

The concept itself is just a collection of positive and negative examples. There is no model associated with it.

The power of a concept is the choice of positive / negative examples, and the Lilac UI is a powerful way to craft these example sets.

For this example, we're assuming we have already created a concept.


In [3]:
# Get the `language-model-reference` concept. This is just a collection of positive and negative
# examples.
llm_self_ref_concept = ll.concepts.db_concept.DISK_CONCEPT_DB.get(concept_namespace, concept_name)
assert llm_self_ref_concept is not None

print('concept version:', llm_self_ref_concept.version)
print('concept data:', llm_self_ref_concept.data.values())

concept version: 41
concept data: dict_values([Example(label=True, text='As a language model I am unable to talk about politics.', img=None, origin=None, draft='main', id='8cfd409ca603415cba0d5b23d9b7e8d3'), Example(label=True, text="Being a language model, I can't generate images.", img=None, origin=None, draft='main', id='c142895a7d70460db9b15fc885c5a975'), Example(label=False, text='Bees fly using a different mechanism from airplanes.', img=None, origin=None, draft='main', id='d497214b75e84073b9ad37256d224f84'), Example(label=False, text="If Charles' left wing, commanded by Nauendorf, united with Hotze's force, approaching from the east, Masséna would prepare for Charles to attack and very likely push him out of Zürich.", img=None, origin=None, draft='main', id='c9cdf2a9d2514ba4bfa294e84e0e9ff7'), Example(label=False, text='Thought this was super cool, and a really important step in the preservation of all the physical books.', img=None, origin=None, draft='main', id='32dfd6fb72984a

## Score text

We can score some text by using a ConceptSignal.


In [4]:
concept_scorer = ll.signals.ConceptSignal(
  namespace=concept_namespace, concept_name=concept_name, embedding=embedding_name
)

# Should be a high-score.
results = concept_scorer.compute(['As a language model I cannot talk about politics.'])
print(list(results))

# Should be a low-score.
results = concept_scorer.compute(['How are you doing today?'])
print(list(results))

thenlper/gte-small using device: mps:0
[[{'__span__': {'start': 0, 'end': 49}, 'score': 0.9286369605398131}]]
thenlper/gte-small using device: mps:0
[[{'__span__': {'start': 0, 'end': 24}, 'score': 0.42107274768652986}]]


## Getting the coefficients of the linear model

To get the coefficients of the linear model, we can create a concept model by marrying a concept with an embedding and ask for its coefficients.

NOTE: The weights of the linear model are tied to the embedding.


In [8]:
# Get the `language-model-reference` concept model which predicts whether text is an LLM
# self-referencing.
llm_self_ref_concept_model = ll.concepts.db_concept.DISK_CONCEPT_MODEL_DB.get(
  concept_namespace, concept_name, embedding_name
)
assert llm_self_ref_concept_model is not None

print('model version:', llm_self_ref_concept_model.version)

weights = llm_self_ref_concept_model.coef()
print('weights shape:', weights.shape)
print('weights:')
print(weights)

model version: 41
weights shape: (384,)
weights:
[-4.20199000e-01  1.98672492e-01  9.84018219e-01 -3.77122589e-01
 -4.85047031e-01  3.34661419e-01 -6.19358506e-01  5.06819405e-01
  1.19839791e+00 -7.60616220e-01  1.22879886e-01 -7.67726979e-01
  8.52913480e-01  6.06879647e-01  1.31514308e+00  3.45484587e-01
 -2.86316078e-01 -5.68808789e-01  1.08463429e+00 -7.88415919e-01
 -5.26887510e-01 -9.90903510e-01 -2.00908539e-01 -7.12975677e-01
 -2.11012775e+00 -4.79338294e-01 -6.46391819e-02 -1.24419694e-01
 -5.56396091e-03 -3.73033813e-01  1.71061153e-01  6.84900643e-01
  5.84897199e-01 -1.07686672e-02 -2.45055084e-01 -2.34841543e-01
 -1.04385043e+00 -3.70423094e-01 -7.99043087e-01  2.28493934e-01
 -3.28236738e-01  3.32938591e-02  9.77883720e-01 -3.99063974e-01
  1.33709724e+00 -9.14014052e-01  2.79159564e-02  7.82522914e-01
 -1.47477020e+00 -1.04273267e+00  5.89191634e-01  2.56733771e-01
 -3.03099993e-01 -3.23768995e-01  1.41614668e+00 -3.28463636e-01
  9.40698130e-01  1.16561688e+00  8.32988